In [1]:
#!/usr/bin/env python3

from blind_rename import rename_imposter_cols

from cimaq_utils import flatten
from cimaq_utils import get_cimaq_dir_paths
from cimaq_utils import loadfiles
from cimaq_utils import loadimages
from cimaq_utils import sortmap

# from inspect_misc_text import filter_lst_exc
# from inspect_misc_text import filter_lst_inc
# from inspect_misc_text import make_labels
# from inspect_misc_text import no_ascii

from inspect_misc_text import *
from inspect_zip import *

from zipctl import uzipfiles
from zipctl import getnamelist
from zipctl import getinfolist
from get_infos import get_infos

from chardet import UniversalDetector as udet
from os.path import expanduser as xpu
from os.path import join as pjoin
from typing import Union
from zipfile import ZipFile

from removeEmptyFolders import removeEmptyFolders
from multiple_replace import multiple_replace
# 'cimaq_derivatives/task_files/uzeprimes'
cimaq_dir= '~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives'

In [2]:
dlst, patterns = get_cimaq_dir_paths(cimaq_dir)

In [3]:
with_encs = [scan_zip_contents(fpath, exclude = ['pratique', 'practice'],
                               to_xtrct = ['.pdf', '.edat2'],
                               withbytes=True,
                               dst_path = dlst.uzeprimes.fpaths)
             for fpath in tqdm(loadimages(dlst.zeprimes.fpaths), desc = 'scanning')]


scanning:   0%|          | 0/103 [00:00<?, ?it/s]
extraction: 3it [00:00, 357.66it/s]

extraction: 2it [00:00, 613.97it/s]

extraction: 2it [00:00, 598.03it/s]
scanning:   3%|▎         | 3/103 [00:00<00:03, 25.30it/s]
extraction: 3it [00:00, 351.19it/s]

extraction: 0it [00:00, ?it/s]

extraction: 3it [00:00, 344.50it/s]
scanning:   6%|▌         | 6/103 [00:00<00:03, 25.39it/s]
extraction: 3it [00:00, 315.20it/s]

extraction: 2it [00:00, 540.43it/s]

extraction: 2it [00:00, 633.87it/s]
scanning:   9%|▊         | 9/103 [00:00<00:03, 25.72it/s]
extraction: 2it [00:00, 645.67it/s]

extraction: 6it [00:00, 840.26it/s]

extraction: 3it [00:00, 451.86it/s]
scanning:  12%|█▏        | 12/103 [00:00<00:03, 25.34it/s]
extraction: 2it [00:00, 420.86it/s]

extraction: 2it [00:00, 618.22it/s]

extraction: 3it [00:00, 542.16it/s]
scanning:  15%|█▍        | 15/103 [00:00<00:03, 25.64it/s]
extraction: 2it [00:00, 693.39it/s]

extraction: 6it [00:00, 740.61it/s]

extraction: 2it [00:00, 668.36it/s]
sca

In [4]:
with_encs[10]

,filename,compress_type,filemode,external_attr,file_size,compress_size,src_name,ext,bsheets
0,3589314_893978_tache_irm_encoding-scan-3589314...,deflate,-rwxr-xr-x,0x4000,206476,12751,3589314_893978_Tache_IRM/Encoding-scan-3589314...,.txt,b'\xff\xfe*\x00*\x00*\x00 \x00h\x00e\x00a\x00d...
1,3589314_893978_tache_irm_onset-event-encoding_...,deflate,-rwxr-xr-x,0x4000,21512,2826,3589314_893978_Tache_IRM/Onset-Event-Encoding_...,.txt,b'1 enc enc00 ...
2,3589314_893978_tache_irm_output-responses-enco...,deflate,-rwxr-xr-x,0x4000,3533,1190,3589314_893978_Tache_IRM/Output-Responses-Enco...,.txt,b'trialnumber\tcategory\ttrialcode\toldnumber\...
3,3589314_893978_tache_irm_output_retrieval_cima...,deflate,-rwxr-xr-x,0x4000,5875,2213,3589314_893978_Tache_IRM/Output_Retrieval_CIMA...,.txt,b'category\tstim\toldnumber\trecognition_acc\t...
4,3589314_893978_tache_irm_retrieval-3589314-1.txt,deflate,-rwxr-xr-x,0x4000,126996,6139,3589314_893978_Tache_IRM/Retrieval-3589314-1.txt,.txt,b'\xff\xfe*\x00*\x00*\x00 \x00h\x00e\x00a\x00d...


In [5]:
sheet_infos = [pd.concat([df(pd.Series({**scan_archv(row[1].bsheets),
                **row[1].to_dict()}))
                for row in itm.iterrows()], axis=1).T.reset_index(drop=True)
               for itm in tqdm(with_encs)]
sheet_infos2 = pd.concat(sheet_infos)

100%|██████████| 103/103 [00:03<00:00, 32.24it/s]


In [6]:
newsheets = [[(row[1].filename, row[1].src_name,
               mkfrombytes(row[1].bsheets, row[1].encoding, row[1].delimiter,
                           row[1].has_header))
             for row in itm.iterrows()] for itm in tqdm(sheet_infos)]

100%|██████████| 103/103 [00:10<00:00,  9.87it/s]


In [82]:
newsheets[10][1][2][0][0]

b'1'

In [37]:
def _update_timestamp(path: os.PathLike, set_new: bool) -> None:
    """
    Context manager to set the timestamp of the path to plus or
    minus a fixed delta, regardless of modifications within the context.

    if set_new is True, the delta is added. Otherwise, the delta is subtracted.
    """
    stats = os.stat(path)
    if set_new:
        new_timestamp = (stats.st_atime_ns + _TIMESTAMP_DELTA, stats.st_mtime_ns + _TIMESTAMP_DELTA)
    else:
        new_timestamp = (stats.st_atime_ns - _TIMESTAMP_DELTA, stats.st_mtime_ns - _TIMESTAMP_DELTA)
    try:
        yield
    finally:
        os.utime(path, ns=new_timestamp)


# Public Methods 